<div align=center>
		
<p></p>
<p></p>
<font size=5>
In the Name of God
<font/>
<p></p>
 <br/>
    <br/>
    <br/>
<font color=#FF7500>
Sharif University of Technology - Departmenet of Computer Engineering
</font>
<p></p>
<font color=blue>
Artifical Intelligence - Dr. Mohammad Hossein Rohban
</font>
<br/>
<br/>
Fall 2021

</div>

<hr/>
		<div align=center>
		    <font color=red size=6>
			    <br />
Practical Assignment 5-Q1
            	<br/>
			</font>
    <br/>
    <br/>
<font size=4>
			<br/><br/>
Deadline:  Bahman 17th
                <br/><b>
              Cheating is Strongly Prohibited
                </b><br/><br/>
                <font color=red>
Please run all the cells.
     </font>
</font>
                <br/>
    </div>

# Personal Data

In [11]:
# Set your student number
student_number = 98102284
Name = 'Parsa'
Last_Name = 'Mohammadian'


# Rules
- You are not allowed to use provided codes that can be found on the internet. 
- If you want to use a library which is not already imported, you must ask a question on Quera to get the permission of using that.
- Do not hesitate to ask questions on Quera, if you have any.
- This assignment is due Bahman 17th 23:59:59. you can use up to 1 grace day for this assignment and the hard deadline is Bahman 18th 23:59:59.

# Q1 (30Points)

## Mountain Car

The OpenAI Gym library includes a set of Python Reinforcement Learning environments. In this question, we examine the Mountain Car environment of this collection. Mountain Car is a Reinforcement Learning task that aims to learn the policy of climbing a steep hill and reaching the flag-marked goal. Also, the car engine is not powerful enough to climb straight up the hill on the right. Therefore, it must gain enough acceleration by climbing the hill on the left.


In this case, the state of the car is determined by an array containing its position and speed.

| Num | Observation | Min | Max|
| --- | --- | --- | ---|
| 0 | Position | -1.3 | 0.5 |
| 1 | Velocity | -0.07 | 0.07|

The intelligent agent is allowed to perform three movements of right push, no push and left push in each step. The agent's move will be given to the environment and the environment returns the next state along with the movement reward. For each step that the car does not reach the target, a cost of -1 is considered. Use Q-learning to find the optimal policy in each case. To do this, you must complete the functions below.

** Note that you will receive the full score of this question only if you can achieve a score of -115 with your implementation.

<font size=4>
Author: Atoosa Chegini
			<br/>
                <font color=red>
Please run all the cells.
     </font>
</font>
                <br/>
    </div>

In [12]:
import numpy as np
import random
import gym


In [13]:
env = gym.make("MountainCar-v0")
# initialization (parameters such as learning rate, gamma, number of states, number of actions, and q_table)
alpha = 0.1
gamma = 0.99
num_states = env.observation_space.shape[0]
num_actions = env.action_space.n
num_episodes = int(2e4)
pos_discretize_size = 36
vel_discretize_size = 16
epsilon = 0.9
q_table = np.zeros((pos_discretize_size*vel_discretize_size, num_actions))
print(f'{alpha=}, {gamma=}, {num_states=}, {num_actions=}, {num_episodes=}, {pos_discretize_size=}, {vel_discretize_size=}, {epsilon=}')
# You may change the inputs of any function as you desire.

alpha=0.1, gamma=0.99, num_states=2, num_actions=3, num_episodes=10000, pos_discretize_size=36, vel_discretize_size=16, epsilon=0.9


Next cell wants you supplement two functions. First for transforming the continuous space into discrete one (in order to make using q_table feasible), second for updating q_values based on the last action done by agent.

In [14]:
# This is just one example of a discretization function. You can change it as you want.
def discretize_state(x, minn, step):
    return int((x - minn) / step)


def env_state_to_Q_state(state):
    [position, velocity] = state
    pos_low, pos_high = -1.2, 0.6
    vel_low, vel_high = -0.07, 0.07
    # Complete this function!
    pos_step = (pos_high - pos_low) / (pos_discretize_size-1)
    vel_step = (vel_high - vel_low) / (vel_discretize_size-1)
    idx1 = discretize_state(position, pos_low, pos_step)
    idx2 = discretize_state(velocity, vel_low, vel_step)
    return idx1*vel_discretize_size + idx2


def update_q(current_state, next_state, action, reward):
    best_future_q = np.max(q_table[next_state])
    # Complete this function!
    q_table[current_state][action] = q_table[current_state][action] + alpha * (reward + gamma * best_future_q - q_table[current_state][action])


In [15]:
# Test desctretization function
assert env_state_to_Q_state([-1.2, -0.07]) == 0
assert env_state_to_Q_state([0.6, 0.07]) == pos_discretize_size*vel_discretize_size-1

At the following cell, the ends of two functions are getting current action based on the policy and defining the training process respectively.

In [16]:
# You may change the inputs of any function as you desire.
from IPython.display import clear_output


def get_action(current_state):
    return np.argmax(q_table[current_state])


def q_learning():
    pass
    # Complete this funciton!
    global epsilon
    for e in range(1, num_episodes+1):
        state, done = env.reset(), False
        while not done:
            random_action = random.randint(0, num_actions-1)
            random_prob = random.random()
            if random_prob < epsilon:
                action = random_action
            else:
                action = get_action(env_state_to_Q_state(state))
            next_state, reward, done, _ = env.step(action)
            q_state = env_state_to_Q_state(state)
            next_q_state = env_state_to_Q_state(next_state)
            update_q(q_state, next_q_state, action, reward)
            state = next_state
        epsilon *= 0.99
        clear_output(wait=True)
        print(f'Episode {e}/{num_episodes} finished.')


def save_policy():
    policy = []
    for item in q_table:
        policy.append(np.argmax(item))
    np.save('policy.npy', policy)


In [17]:
q_learning()
save_policy()


Episode 10000/10000 finished.


In [18]:
save_policy()
np.load("policy.npy")
assert not np.all(np.load("policy.npy") == 0)
np.count_nonzero(np.load("policy.npy"))

243

In [19]:
# Attention: don't change this function. we will use this to grade your policy which you will hand in with policy.npy
# btw you can use it to see how you are performing. Uncomment two lines which are commented to be able to see what is happening visually.
def score():
    policy, scores = np.load("policy.npy"), []
    print(policy)
    for episode in range(1000):
        # print(f"******Episode {episode}")
        state, score, done, step = env_state_to_Q_state(
            env.reset()), 0, False, 0
        while not done:
            # time.sleep(0.04)
            action = policy[state]
            state, reward, done, _ = env.step(action)
            state = env_state_to_Q_state(state)
            step += 1
            score += int(reward)
            # env.render()
        # print(f"Score:{score}")
        scores.append(score)
    print(f"Average score over 1000 run : {np.array(scores).mean()}")


score()


[0 1 0 1 0 0 2 2 2 1 0 0 0 0 0 0 0 1 2 1 0 0 1 2 2 2 2 0 0 0 0 0 0 0 2 0 0
 2 2 2 2 2 1 0 0 0 0 0 0 0 0 0 1 1 2 2 2 1 2 2 0 0 0 0 0 0 2 2 1 0 0 2 2 2
 2 2 0 0 0 0 2 0 1 1 0 0 2 1 2 2 2 2 0 0 0 0 2 0 1 1 0 0 2 2 1 0 2 2 2 0 0
 0 1 0 0 0 2 0 0 2 1 0 2 2 2 0 0 0 0 2 0 0 0 0 0 2 2 2 1 1 2 0 0 0 0 1 0 0
 0 0 2 2 2 1 0 2 2 1 0 0 0 0 0 0 0 0 2 2 2 2 1 2 1 1 0 0 1 0 0 0 0 0 0 0 2
 1 0 2 2 2 0 0 0 0 1 0 0 0 0 0 2 1 2 0 2 1 0 0 2 1 1 1 0 1 0 0 2 2 0 2 2 2
 0 0 2 2 1 0 0 0 0 0 2 2 0 2 2 2 0 0 0 2 0 0 0 0 0 0 2 2 1 2 2 2 0 0 1 0 0
 1 1 0 0 0 2 2 0 1 2 2 0 0 1 2 1 0 0 0 1 0 2 0 1 0 2 2 0 0 0 1 1 0 1 0 0 0
 2 2 1 0 0 2 0 0 0 1 0 2 0 0 0 1 0 0 1 2 2 2 0 0 0 0 0 2 2 0 0 0 1 1 0 2 2
 2 0 0 0 2 0 1 2 0 0 0 1 1 1 2 2 2 0 0 0 0 0 2 1 0 1 2 0 0 1 2 0 2 0 0 0 0
 0 0 2 1 2 1 0 0 0 2 2 1 0 0 0 0 1 0 0 1 2 0 1 1 2 1 2 0 0 0 0 0 0 1 0 0 0
 0 1 2 2 1 0 0 0 0 0 0 0 2 0 1 2 0 2 2 2 2 2 0 0 0 0 0 0 0 1 0 1 1 2 2 2 2
 2 0 0 0 0 0 0 0 2 0 0 0 2 2 2 2 2 0 0 0 0 0 0 0 0 0 0 0 2 1 2 2 2 0 0 0 0
 0 0 0 0 1 2 0 2 2 1 2 2 